## Open-Meteo Air Quality: Là dữ liệu mô hình (modelled) từ mô hình CAMS (Copernicus Atmosphere Monitoring Service) của Châu Âu. Nó mô phỏng lại nồng độ ô nhiễm trên một lưới toàn cầu.

### data chỉ có từ năm 2022

In [1]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 1: Khởi tạo môi trường

import pandas as pd
import os
import glob
import openmeteo_requests
import requests_cache
from retry_requests import retry

print("Cell 1: Môi trường cho việc thu thập dữ liệu Air Quality (CAMS) đã sẵn sàng.")

Cell 1: Môi trường cho việc thu thập dữ liệu Air Quality (CAMS) đã sẵn sàng.


In [2]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 2: Đọc file Metadata

metadata_filename = "../weatherData/stations_metadata.csv"

if not os.path.exists(metadata_filename):
    print(f"LỖI: File '{metadata_filename}' không tồn tại.")
    print("Vui lòng chạy Cell 2.5 trong notebook OpenAQ trước để tạo file này.")
else:
    df_metadata = pd.read_csv(metadata_filename)
    # Chuyển đổi cột ngày tháng sang kiểu datetime để so sánh
    df_metadata['start_date'] = pd.to_datetime(df_metadata['start_date'], utc=True)
    df_metadata['end_date'] = pd.to_datetime(df_metadata['end_date'], utc=True)
    
    print(f"Đã đọc thành công thông tin của {len(df_metadata)} trạm từ '{metadata_filename}'.")

Đã đọc thành công thông tin của 31 trạm từ '../weatherData/stations_metadata.csv'.


In [3]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 3: Tải và lưu các file Air Quality Raw

# Kiểm tra xem df_metadata đã được đọc chưa
if 'df_metadata' not in locals():
    print("Vui lòng chạy Cell 2 trước để đọc file metadata.")
else:
    # 1. Thiết lập API client
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # 2. Tạo thư mục chính để chứa dữ liệu
    aq_output_dir = "hanoi_air_quality_cams_raw"
    if not os.path.exists(aq_output_dir):
        os.makedirs(aq_output_dir)
        print(f"Đã tạo thư mục: {aq_output_dir}")

    print(f"\nBắt đầu quá trình tải dữ liệu Air Quality cho {len(df_metadata)} vị trí...")

    # 3. Lặp qua từng trạm
    for index, station in df_metadata.iterrows():
        loc_id = station['location_id']
        lat = station['lat']
        lon = station['lon']
        
        start_date_cams = max(station['start_date'], pd.to_datetime("2022-01-01T00:00:00Z", utc=True))
        end_date_cams = station['end_date']

        if start_date_cams > end_date_cams:
            print(f"\n--- Trạm ID: {loc_id} không có dữ liệu trong khoảng thời gian của CAMS (từ 2022). Bỏ qua. ---")
            continue
            
        start_date_str = start_date_cams.strftime('%Y-%m-%d')
        end_date_str = end_date_cams.strftime('%Y-%m-%d')
        
        print(f"\n--- Đang lấy dữ liệu Air Quality cho vị trí của trạm ID: {loc_id} từ {start_date_str} đến {end_date_str} ---")

        # 4. Xây dựng tham số và gọi API
        url = "https://air-quality-api.open-meteo.com/v1/air-quality"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date_str,
            "end_date": end_date_str,
            "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "dust"],
        }
        
        try:
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            
            # 5. Xử lý dữ liệu trả về và tạo DataFrame
            hourly = response.Hourly()
            hourly_data = {"datetime": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )}

            variables = params["hourly"]
            for i, var_name in enumerate(variables):
                values = hourly.Variables(i).ValuesAsNumpy()
                hourly_data[f"{var_name}_cams"] = values[:len(hourly_data["datetime"])]

            df_aq_cams = pd.DataFrame(data=hourly_data)
            
            # 6. Lưu file CSV riêng lẻ
            output_filename = os.path.join(aq_output_dir, f"air_quality_cams_{loc_id}.csv")
            df_aq_cams.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"  -> Đã lấy và lưu thành công {len(df_aq_cams)} giờ dữ liệu vào file {output_filename}")

        except Exception as e:
            print(f"  -> Đã xảy ra lỗi khi lấy dữ liệu cho vị trí trạm {loc_id}: {e}")

    print("\n\n---> HOÀN TẤT QUÁ TRÌNH TẢI DỮ LIỆU AIR QUALITY (CAMS) <---")
    print(f"Dữ liệu thô đã được lưu vào các file riêng lẻ trong thư mục '{aq_output_dir}'")

Đã tạo thư mục: hanoi_air_quality_cams_raw

Bắt đầu quá trình tải dữ liệu Air Quality cho 31 vị trí...

--- Trạm ID: 2539 không có dữ liệu trong khoảng thời gian của CAMS (từ 2022). Bỏ qua. ---

--- Đang lấy dữ liệu Air Quality cho vị trí của trạm ID: 7441 từ 2022-01-01 đến 2025-04-09 ---
  -> Đã lấy và lưu thành công 28680 giờ dữ liệu vào file hanoi_air_quality_cams_raw\air_quality_cams_7441.csv

--- Trạm ID: 1285357 không có dữ liệu trong khoảng thời gian của CAMS (từ 2022). Bỏ qua. ---

--- Đang lấy dữ liệu Air Quality cho vị trí của trạm ID: 2161290 từ 2024-01-29 đến 2025-06-10 ---
  -> Đã lấy và lưu thành công 11976 giờ dữ liệu vào file hanoi_air_quality_cams_raw\air_quality_cams_2161290.csv

--- Đang lấy dữ liệu Air Quality cho vị trí của trạm ID: 2161291 từ 2024-01-22 đến 2024-12-11 ---
  -> Đã lấy và lưu thành công 7800 giờ dữ liệu vào file hanoi_air_quality_cams_raw\air_quality_cams_2161291.csv

--- Đang lấy dữ liệu Air Quality cho vị trí của trạm ID: 2161292 từ 2024-01-29 đến

In [4]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 4: Gộp các file Air Quality Raw

aq_output_dir = "hanoi_air_quality_cams_raw"

if not os.path.exists(aq_output_dir):
    print(f"Thư mục '{aq_output_dir}' không tồn tại. Vui lòng chạy Cell 3 trước.")
else:
    # 1. Tìm tất cả các file .csv trong thư mục
    all_csv_files = glob.glob(os.path.join(aq_output_dir, "*.csv"))
    
    if not all_csv_files:
        print("Không tìm thấy file CSV nào để gộp.")
    else:
        print(f"Tìm thấy {len(all_csv_files)} file CAMS CSV. Bắt đầu gộp...")
        
        # 2. Đọc, thêm location_id và gộp tất cả các file
        all_data_frames = []
        for f in all_csv_files:
            df = pd.read_csv(f)
            # Lấy location_id từ tên file
            loc_id = int(os.path.basename(f).replace('air_quality_cams_', '').replace('.csv', ''))
            df['location_id'] = loc_id
            all_data_frames.append(df)
            
        final_df = pd.concat(all_data_frames, ignore_index=True)
        
        # 3. Xử lý và sắp xếp lần cuối
        final_df['datetime'] = pd.to_datetime(final_df['datetime'])
        final_df = final_df.sort_values(by=['location_id', 'datetime'])
        
        # 4. Lưu file tổng hợp cuối cùng
        final_output_filename = "hanoi_air_quality_CAMS_COMBINED.csv"
        final_df.to_csv(final_output_filename, index=False, encoding='utf-8-sig')
        
        print("\n---> QUÁ TRÌNH GỘP HOÀN TẤT <---")
        print(f"Tổng cộng có {len(final_df)} dòng dữ liệu.")
        print(f"Dữ liệu CAMS đã được gộp và lưu vào file '{final_output_filename}'")
        print("\nThông tin chi tiết của DataFrame cuối cùng:")
        final_df.info()

Tìm thấy 29 file CAMS CSV. Bắt đầu gộp...

---> QUÁ TRÌNH GỘP HOÀN TẤT <---
Tổng cộng có 159768 dòng dữ liệu.
Dữ liệu CAMS đã được gộp và lưu vào file 'hanoi_air_quality_CAMS_COMBINED.csv'

Thông tin chi tiết của DataFrame cuối cùng:
<class 'pandas.core.frame.DataFrame'>
Index: 159768 entries, 131088 to 131087
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   datetime               159768 non-null  datetime64[ns, UTC]
 1   pm10_cams              154608 non-null  float64            
 2   pm2_5_cams             154608 non-null  float64            
 3   carbon_monoxide_cams   154608 non-null  float64            
 4   nitrogen_dioxide_cams  154608 non-null  float64            
 5   sulphur_dioxide_cams   154608 non-null  float64            
 6   ozone_cams             154608 non-null  float64            
 7   dust_cams              154608 non-null  float64            
 8   l